In [2]:
%pip install particle
%pip install sympy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Lets start by defining a decay. In the code beow you can see how to define a simple utility, which given momenta will be able to claculate all needed rotations.

For simplicity we will choose a simple
As you see, the isobars are allways isolated, meaning no isobar is the predecessor of another isobar. This makes the the full amplitude straightforward to calculate.

In [3]:
from decayangle.DecayTopology import TopologyGroup, Node
import numpy as np

# Find the possible the decay topologies
tg = TopologyGroup(0, [1,2,3])
for decay_tree in tg.trees:
    print(decay_tree)

( 0 -> 1, ( (2, 3) -> 2, 3 ) )
( 0 -> 2, ( (1, 3) -> 1, 3 ) )
( 0 -> ( (1, 2) -> 1, 2 ), 3 )


First lets define some basic utility functions. This is not something you need to understand 100% on the first try.
What we do is set up the basic functional setup to be able to perform a three body analysis, which is the simplest case. 

In [4]:
from typing import NamedTuple, Callable
from math import prod
from particle import Particle


def wigner_small_d(theta, j, m1, m2):
    """Calculate Wigner small-d function. Needs sympy.
      theta : angle
      j : spin (in units of 1/2, e.g. 1 for spin=1/2)
      m1 and m2 : spin projections (in units of 1/2)

    :param theta:
    :param j:
    :param m1: before rotation
    :param m2: after rotation

    """
    from sympy import Rational
    from sympy.abc import x
    from sympy.utilities.lambdify import lambdify
    from sympy.physics.quantum.spin import Rotation as Wigner
    j,m1,m2 = int(j),int(m1),int(m2)
    # TODO: check if this is correct (the order of the m1 and m2)
    d = Wigner.d(Rational(j, 2), Rational(m2, 2), Rational(m1, 2), x).doit().evalf()

    return lambdify(x, d, "jax")(theta)

    
def BWResonance(spin, mass, width):
    """Create a Breit-Wigner resonance function for a given spin.
    Args:
        spin (int): spin quantum number multiplied by 2
    """
    spin_config = list(range(-spin, spin+1, 2))
    def f(s, h0, h1, h2, psi_rf, theta_rf):
        return wigner_small_d(theta_rf, spin, h0, h1 - h2) * np.sqrt(s) / (s - mass**2 + 1j * mass * width)
    
    return f

class resonance:
    def __init__(self, spin, parity, mass, width, name, mother:Particle, bachelor:Particle, isobar: tuple):
        self.spin = spin
        self.mass = mass
        self.width = width
        self.lineshape = BWResonance(spin, mass, width)
        self.parity = parity
        self.name = name
        self.mother = mother
        self.bachelor = bachelor
        self.isobar = isobar


    @property
    def possible_helicities(self):
        return list(range(-self.spin, self.spin+1, 2))
    
    @property
    def isbar_spins(self):
        return [p.spin for p in self.isobar]
    
    def isobar_parities(self):
        return [p.parity for p in self.isobar]

    def LS_couplings_mother_decay(self):
        allowed_ls = []
        LSCoupling = NamedTuple('LSCoupling', [('L', int), ('S', int), ('coupling', complex)])
        for L in range(0, self.mother.spin + self.spin + self.bachelor.spin + 1, 2):
            for S in range(abs(L - self.mother.spin), L + self.mother.spin + 1, 2):
                if abs(L - S) <= self.mother_spin <= L + S:
                    allowed_ls.append((L, S))
        return {
            f"{self.name}:motherDecay:L{L}.S{S}": LSCoupling(L, S, 0 + 0j) for L, S in allowed_ls
        }
    
    def LS_coupling_resonance_decay(self):
        allowed_ls = []
        LSCoupling = NamedTuple('LSCoupling', [('L', int), ('S', int), ('coupling', complex)])
        for L in range(0, self.spin + sum(self.isbar_spins), 2):
            for S in range(abs(L - self.spin), L + self.spin + 1, 2):
                if abs(L - S) <= self.spin <= L + S: # check J double
                    if prod(self.isobar_parities) * (-1)**(L) == self.parity: # check parity
                        allowed_ls.append((L, S))
        return {
            f"{self.name}:resonanceDecay:L{L}.S{S}": LSCoupling(L, S, 0 + 0j) for L, S in allowed_ls
        }
    
    def h(self, LSmother_decay:dict, LSresonance_decay:dict) -> float:
        # TODO: calculate the h values from the LS values and  clebsch gordans
        return 1.

        

possible_helicities = lambda spin: list(range(-spin, spin+1, 2))

In [5]:
# shortcut to get the pdg parameters for our particles
class particle:
    def __init__(self, p):
        self.p = p 
    
    @property
    def spin(self):
        return int(self.p.J * 2)

    @property
    def parity(self):
        return self.p.P

    def __getattr__(self, attr):
        return getattr(self.p, attr)

p0 = particle(Particle.findall('Lambda(b)0')[0])
p1 = particle(Particle.findall('D0')[0])
p2 = particle(Particle.findall('Lambda(c)+')[0])
p3 = particle(Particle.findall('K-')[0])

particles = {
    0: p0,
    1: p1,
    2: p2,
    3: p3
}


Now we have the main components for the amplitude. We can describe the lineshape for as single resonance, we can compute helicity couplings and we can calculate all the angles. Only thing left to do is write the loops for the sums. The final form should look somewhat like this:

$\sum_{\mathrm{trees}}\sum_{\{\lambda\}}\sum_{\{\lambda^{'}\}} R(m,\{\lambda^{'}\}) d(\theta, \lambda_k^{'}, \lambda_{ij}^{'}) d(\phi_1,\lambda^{'}_1,\lambda_1)d(\phi_2,\lambda^{'}_2,\lambda_2)d(\phi_3,\lambda^{'}_3,\lambda_3)$

This simply describes the sum over all different possible decay trees, and the sum over all possible helicities.
A second sum over the primes helicities $\{\lambda^{'}\}$ in conjuction with the wigner roatation $d(\theta, \lambda_k^{'}, \lambda_{ij}^{'})$ describes the needed basis change after the rotation to move the $(ij)$ isobar onto the z axis.

The rotations $d(\phi_1,\lambda^{'}_1,\lambda_1)d(\phi_2,\lambda^{'}_2,\lambda_2)d(\phi_3,\lambda^{'}_3,\lambda_3)$ describe the rotations rotating the final states of the different chains to a common constellation.

The $R(m,\{\lambda^{'}\})$ is the lineshape of the resonance together with the couplings $H_{0 \rightarrow (ij)\, k}^{\lambda_0, \lambda_{(ij) - \lambda_k}} \cdot X(m,L) \cdot H_{(ij) \rightarrow i\, j}^{\lambda_{(ij)}, \lambda_i^{'} - \lambda_j^{'}}$. 
The couplings $H$ are alled helicity couplings 

In [6]:
momenta = { 
    1: np.array([0, 0, -0.9, 1]),
    2: np.array([0, 0.15, 0.4,1]),
    3: np.array([ 0, 0.3, 0.3,1]),
}

reference_tree = tg.trees[0]
momenta = reference_tree.to_rest_frame(momenta)

# We can define resonances based on the isobar they can appear in 
resonance_lineshapes = {
    (1, 3): resonance(2, 1, 2800, 100, 'D2800', p0, p2, (p1, p3)), 
                        }


Now we need to get the values for our rotations. For this we can simply iterate through all the possible decay trees, the ```TopologyGroup``` Object generates for us.

In [9]:
for tree in tg.trees:
        final_state_rotations = {
            target:reference_tree.relative_wigner_angles(tree, target, momenta)
            for target in [1, 2, 3]
        }
        isobars = tree.helicity_angles(momenta)
        print(tree, isobars)

( 0 -> 1, ( (2, 3) -> 2, 3 ) ) {(2, 3): (3.141592653589793, -1.1917396523105919)}
( 0 -> 2, ( (1, 3) -> 1, 3 ) ) {(1, 3): (5.951828758370096e-17, -0.23908244819164334)}
( 0 -> ( (1, 2) -> 1, 2 ), 3 ) {(1, 2): (5.041182631349134e-17, -0.26891484091188184)}


/home/kai/LHCb/decayangle/src/decayangle/kinematics.py:214: RuntimeWarning: invalid value encountered in divide
  cosine_input = config.backend.where(abs(abs_mom) <= 1e-19, 0, z_component(V) / abs_mom)


For each tree, we see, that we get exactly one isobar constellation. We also see, that of the two angles we compute, only one is non 0 (or $\pi$).

This is expected, as we are already in the decay plane. Thus this angle should go to 0. Computing it could be left out.

In [8]:
# particle 1 
spin1 = 1
helicities1 = possible_helicities(spin1)

# particle 2
spin2 = 1
helicities2 = possible_helicities(spin2)

# particle 3
spin3 = 0
helicities3 = possible_helicities(spin3)


# we can use the tree objects to controll out sums. 
# This makes the function definition quite simple
def f(h0, h1, h2, h3):
    for tree in tg.trees:
        final_state_rotations = {
            target:reference_tree.relative_wigner_angles(tree, target, momenta)
            for target in [1, 2, 3]
        }
        isobars = tree.helicity_angles(momenta)
        
        print(isobars)

f(1, 1, 0, 0)
        

{(2, 3): (3.141592653589793, -1.1917396523105919)}
{(1, 3): (5.951828758370096e-17, -0.23908244819164334)}
{(1, 2): (5.041182631349134e-17, -0.26891484091188184)}
